In [5]:
####GATE DECOMPOSITION#####
'''
h: rz(pi/2) rx(pi/2) rz(pi/2)
y: rx(pi)rz(pi)
ry(theta): rz(-pi/2) rx(theta) rz(pi/2)
cx :  rz(pi/2)[q1] rx(pi/2)[q1] rz(pi/2)[q1] cz[q0][q1] rz(pi/2)[q1] rx(pi/2)[q1] rz(pi/2)[q1]
'''
###########################
gate_map = {'h': 'rz(pi/2)' + '\n' + 'rx(pi/2)' + '\n' + 'rz(pi/2)', 'x': 'rx(pi)', 'y': 'rx(pi)rz(pi)', 'z': 'rz(pi)',
        'rz': 'rz','rx': 'rx','i' : 'rx(0)','cz':'cz', 'u1': 'rz'}


def get_native_ry(QASM_gates,qubits):
    theta = QASM_gates.split('(')[1].split(')')[0]
    return ['rz(-pi/2)',  'rx' + '(' + str(theta) + ')' , 'rz(pi/2)']

def get_native_cx(QASM_gates,qubits):
    q0 = qubits.split(']')[0] + ']'
    q1 = qubits.split(',')[1]
    
    h = 'rz(pi/2) ' + q1,'rx(pi/2) '+  q1 , 'rz(pi/2) ' + q1
    cx = ['rz(pi/2) ' + q1,'rx(pi/2) '+  q1,'rz(pi/2) ' + q1,'cz ' + q0 + ','+ q1  , 'rz(pi/2) ' + q1,'rx(pi/2) '+  q1,'rz(pi/2) ' + q1]
    return cx

def get_native_gates(QASM_gates,qubits):
    if gate_map.get(QASM_gates,None) != None:
        gates = gate_map[QASM_gates].split('\n')
        native_gates = []
        for gate in gates:
            native_gates.append(str(gate))
            
        return native_gates
    
    elif QASM_gates.split('(')[0] == 'ry':
        return get_native_ry(QASM_gates,qubits)
    elif  QASM_gates.split(' ')[0] == 'cx':
        return get_native_cx(QASM_gates,qubits)
    else:
        print('error - native gate not found for',QASM_gates)
        
output_file = open("output_deutsch.qasm", "w+")
        
with open('deutsch.qasm') as f:
    
    for line in range(6):
        output_file.write(f.readline())
        
    datafile = f.readlines()
    for line in datafile:
        if 'measure' in line:
            output_file.write('measure q -> c;')
            break
        else:
            QASM_gate,qubits = line.split(' ')
            
        if 'rx' in QASM_gate:
            output_file.write(QASM_gate +' '+ qubits)
        elif 'rz' in QASM_gate:
            output_file.write(QASM_gate + ' '+qubits)
        elif 'u1' in QASM_gate:
            output_file.write(QASM_gate.replace('u1','rz',1) + ' '+qubits)
        elif 'cx' in QASM_gate:
            native_cx_gates = get_native_cx(QASM_gate,qubits)
            for gate in native_cx_gates:
                output_file.write(gate)
        else:
            native_gates = get_native_gates(QASM_gate,qubits)
            for gate in native_gates:
                output_file.write(gate+' '+qubits)
          
           
output_file.close()
    

Input:3qubit QFT QASM code
output: compiled QASM code with restricted gate set


Tested on IBM Q Experience:https://quantum-computing.ibm.com/composer/new-experiment?initial=N4IgdghgtgpiBcIDCB7KAHAlgGxgEwH0BFAMQBUQAaEARwgGcoEQB5ABQFEA5IgQQGUAsgAIATADoADAG4AOmExgAxtgCueGMNm0Y2TACMAjOMVLtcsPJoAnGAHNhNANoBWALoWlth0tcf58tYAXgAUWAD0ogCUjk6S-mDWAB5hmJExzvEWwanpsVmBoREALBlxCTkR0bGGFSlVZbXZRWnVzk3ySkH5bpTtFS15-dn1rY0DuW1OHYmhALQl482TS4GjQ9MTDTUJXT19m8vbw2srO0dj54W5pVezZyeJ61MzlZePb%2BEAHGUF98dOUR1B6ArbvUGebqZXrOIEXDZw04AxH-cEonILNI-WLowZTdHPMq4kEovbQg7E5HAqnw-Fg75E%2Bm3D54xkjEn0umQnYUzlspFovk4iaY8LMiHXGkChFC2G7KGbXm0-lPDnK4Xw8WUwXsqWomUWWAMVS2RzCOYAPmESmkVBAGnoXkw6AALpgUGBmCAAL5AA

Input:Deutsch-Jozsa QASM code with two qubits for f(x)=x
output: compiled QASM code with restricted gate set.


Tested on IBM Q Experience: https://quantum-computing.ibm.com/composer/new-experiment?initial=N4IgdghgtgpiBcICqYAuBLVAbGATABAMboBOhArpiADQgCOEAzlAiAPIAKAogHICKAQQDKAWXwAmAHQAGANwAdMOjCEs5XDHzz6MLOgBGARknLC2hWEV0SMAOb46AbQCsAXQuEb9wi-eLFJAAeABQADugAlA6OACx%2BYCQAXmHoAPTiUU4AzPFBKemZjjkWSfkZ0cUByeEF0XElITXlTvVVZYWtCdVpzbG5jT0dud21LfGEiRWu1GMlI72deU1Dc%2B11w2vZ-ZtFG8tTFrBM5DYO%2BAC0AHxEsjQgGoye6KEYAPZgrCAAvkA
    

Analysis: ->Routing / Mapping circuits on to physical devices
          ->optimizing gate scheduling such as lookahead/stochastic scheduling to reduce SWAP operations 
          ->Substitute less erroneous gates for more error prone operations
            
    Number of gates and hence depth of the circuit increases with the restricted gate set. Due to low Coherence time in
    the current NISQ systems, qubits cannot hold the information for long time.optimal mapping of circuits to physical devices can reduce the error rates.
    SWAP operations are time consuming and erroneous. various mapping techniques are used to map circuits depending upon the quit connectivity.
    These architecural decisions help the hardware vendors to design processors with more connectivity and avoid bottle neck nodes.
    

References:https://github.com/Qiskit/openqasm/blob/master/examples/ibmqx2/Deutsch_Algorithm.qasm
